In [1]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

def fetch_div_relative(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=1.5)
        response.raise_for_status()  # Raises an HTTPError for bad responses
    except requests.exceptions.Timeout:
        print(f"Request timed out for URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    div_relative_content = soup.find('div', class_='relative')
    return div_relative_content

def extract_tables(div_relative_content):
    tables = div_relative_content.find_all('table')
    return tables

In [36]:
# URL of the webpage to scrape
url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2024May28.HTM"
div_relative_content = fetch_div_relative(url)
# print("Div relative content:", div_relative_content)

if div_relative_content:
    tables = extract_tables(div_relative_content)
    print(type(tables[8]))
    print("Table 1:", tables[8])
    # for table in tables:
    #     print("Table:", table)


<class 'bs4.element.Tag'>
Table 1: <table cols="21" style="margin-right:7%">
<tr style="margin-bottom:-16px">
<td class="datacolumn1">Daniel Phillips</td>
<td class="gameswlcolumn">12</td>
<td class="gameswlcolumn">1</td>
<td class="beforegw"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn1"></td>
<td class="gamesemcolumn">_</td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59251&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59249&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59247&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59245&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
<td class="aftergw"></td>
<td class="racolumn1"></td>
<td class="racolumn1"></td>
<t

In [5]:


# Extract data
data = []
rows = tables[0].find_all('tr')
for row in rows:
    cols = row.find_all('td')
    if cols:
        name = cols[0].get_text(strip=True)
        scores = [col.get_text(strip=True) for col in cols[7:11]]
        ratings = [col.get_text(strip=True) for col in cols[15:19]]
        opponents = [rows[i].find('td').get_text(strip=True) for i in range(len(rows)) if i != rows.index(row)]
        for opponent, score, rating in zip(opponents, scores, ratings):
            if score != "_" and rating != "_":
                data.append([name, opponent, score, rating])

# Create a DataFrame and save it to a CSV file
df = pd.DataFrame(data, columns=['Winner', 'Loser', 'Score', 'Rating Change'])
df


,Winner,Loser,Score,Rating Change
0,Hari Teja Narayanabhatla,Geetha Krishna,3,8
1,Hari Teja Narayanabhatla,Sunil Bandapally,3,4
2,Medha Krishna,Hari Teja Narayanabhatla,2,-8
3,Medha Krishna,Sunil Bandapally,3,4
4,Geetha Krishna,Hari Teja Narayanabhatla,2,-4
5,Geetha Krishna,Medha Krishna,0,-4
6,Sunil Bandapally,Hari Teja Narayanabhatla,1,-3
7,Sunil Bandapally,Medha Krishna,0,-3
8,Sunil Bandapally,Geetha Krishna,0,-7


#### This is the inner loop test to get the score and the rating

In [6]:
rows = tables[0].find_all('tr')
cols = rows[0].find_all('td')

scores_indices = [7, 8, 9, 10]
# Extract how many games the current player won
scores = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).isdigit() else math.nan for i in scores_indices]

ratings_indices = [15, 16, 17, 18]
# Extract the ratings change of the current player
ratings = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).isdigit() else math.nan for i in ratings_indices]

ratings


[nan, 8, 4, 3]

In [23]:
# Extract data
players_data = []
scores_data = []
ratings_data = []

for row in rows:
    cols = row.find_all('td')

    players_data.append(cols[0].get_text(strip=True))

    scores_indices = [7, 8, 9, 10]
    # Extract how many games the current player won
    scores = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).lstrip("-").isdigit() else math.nan for i in scores_indices]

    ratings_indices = [15, 16, 17, 18]
    # Extract the ratings change of the current player
    ratings = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).lstrip("-").isdigit() else math.nan for i in ratings_indices]

    scores_data.append(scores)
    ratings_data.append(ratings)

print("players_data: ", players_data)
print("scores_data: ", scores_data)
print("ratings_data: ", ratings_data)


players_data:  ['Hari Teja Narayanabhatla', 'Medha Krishna', 'Geetha Krishna', 'Sunil Bandapally']
scores_data:  [[nan, 3, 3, 3], [2, nan, 3, 3], [2, 0, nan, 3], [1, 0, 0, nan]]
ratings_data:  [[nan, 8, 4, 3], [-8, nan, 4, 3], [-4, -4, nan, 7], [-3, -3, -7, nan]]


In [21]:
data = []
df = pd.DataFrame(columns=['Winner', 'Loser', 'Winner_Score', 'Loser_Score', 'Rating Change'])

for i in range(len(players_data)):
    for j in range(i + 1, len(players_data)):
        if i != j:
            data.append([players_data[i], players_data[j], scores_data[i][j], scores_data[j][i], ratings_data[i][j]])

data

[['Hari Teja Narayanabhatla', 'Medha Krishna', 3, 2, 8],
 ['Hari Teja Narayanabhatla', 'Geetha Krishna', 3, 2, 4],
 ['Hari Teja Narayanabhatla', 'Sunil Bandapally', 3, 1, 3],
 ['Medha Krishna', 'Geetha Krishna', 3, 0, 4],
 ['Medha Krishna', 'Sunil Bandapally', 3, 0, 3],
 ['Geetha Krishna', 'Sunil Bandapally', 3, 0, 7]]

### Combine everything together

In [40]:
def extract_players_data(inner_tables):
    # Extract data
    players_data = []
    scores_data = []
    ratings_data = []

    for inner_table in inner_tables:
        cols = inner_table.find_all('td')

        players_data.append(cols[0].get_text(strip=True))

        # changes the indices to match the table depending on the number of players in the table
        num_players = len(inner_table.find_all('td', class_='gamesemcolumn'))
        if (num_players == 3):
            scores_indices = [7, 8, 9]
            ratings_indices = [14, 15, 16]
        elif (num_players == 4):
            scores_indices = [7, 8, 9, 10]
            ratings_indices = [15, 16, 17, 18]
        elif (num_players == 5):
            scores_indices = [7, 8, 9, 10, 11]
            ratings_indices = [16, 17, 18, 19, 20]
        else:
            raise Exception("Number of players in the table is not 3, 4, or 5")
        # Extract how many games the current player won
        scores = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).lstrip("-").isdigit() else math.nan for i in scores_indices]

        # Extract the ratings change of the current player
        ratings = [int(cols[i].get_text(strip=True)) if cols[i].get_text(strip=True).lstrip("-").isdigit() else math.nan for i in ratings_indices]

        scores_data.append(scores)
        ratings_data.append(ratings)
    
    return players_data, scores_data, ratings_data


# URL of the webpage to scrape
url = "http://www.tabletennisleague.com/AGTTA/SessionGroupReportArchive/SessionGroupReport2024May28.HTM"
div_relative_content = fetch_div_relative(url)

if div_relative_content:
    tables = extract_tables(div_relative_content)



session_data = []

for table_num, table in enumerate(tables):
    table_num += 1
    inner_tables = table.find_all('tr')
    
    players_data, scores_data, ratings_data = extract_players_data(inner_tables)

    for i in range(len(players_data)):
        for j in range(i + 1, len(players_data)):
            if i != j:
                try:
                    session_data.append([players_data[i], players_data[j], scores_data[i][j], scores_data[j][i], abs(ratings_data[i][j]), table_num])
                except IndexError:
                    print("players_data:", players_data)
                    print("scores_data:", scores_data)
                    print("IndexError:", i, j, "table number:", table_num)
                    # print("session_data:", session_data)

session_data

[['Sunil Bandapally', 'Geetha Krishna', 3, 2, 8, 1],
 ['Sunil Bandapally', 'Andrew Dyl', 3, 0, 0, 1],
 ['Sunil Bandapally', 'Aneece Khalek', 3, 0, 2, 1],
 ['Geetha Krishna', 'Andrew Dyl', 3, 1, 0, 1],
 ['Geetha Krishna', 'Aneece Khalek', 3, 0, 2, 1],
 ['Andrew Dyl', 'Aneece Khalek', 3, 2, 30, 1],
 ['Ermias Aberra', 'Bethany Zhou', 3, 0, 20, 2],
 ['Ermias Aberra', 'Rahul Movva', 3, 0, 7, 2],
 ['Ermias Aberra', 'Michael Yavnel', 3, 2, 2, 2],
 ['Bethany Zhou', 'Rahul Movva', 3, 0, 3, 2],
 ['Bethany Zhou', 'Michael Yavnel', 3, 1, 0, 2],
 ['Rahul Movva', 'Michael Yavnel', 3, 2, 3, 2],
 ['Jon Gustavson', 'Xun Liang', 3, 1, 2, 3],
 ['Jon Gustavson', 'Zorain Malik', 3, 0, 1, 3],
 ['Jon Gustavson', 'Christopher Sester', nan, nan, 0, 3],
 ['Xun Liang', 'Zorain Malik', 3, 1, 7, 3],
 ['Xun Liang', 'Christopher Sester', nan, nan, 0, 3],
 ['Zorain Malik', 'Christopher Sester', nan, nan, 0, 3],
 ['Samrat Bandapally', 'Prabhas Lanka', 3, 2, 13, 4],
 ['Samrat Bandapally', 'Nick Gentry', 3, 0, 16, 4],
 

## Test Cases

In [38]:
table.find_all('tr')

[<tr style="margin-bottom:-16px">
 <td class="datacolumn1">Isha Nitturu</td>
 <td class="gameswlcolumn">9</td>
 <td class="gameswlcolumn">2</td>
 <td class="beforegw"></td>
 <td class="gamesemcolumn1"></td>
 <td class="gamesemcolumn1"></td>
 <td class="gamesemcolumn">_</td>
 <td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59260&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
 <td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59253&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
 <td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59256&amp;sessionnumber=7" title="Click to view details of this match">3</a></td>
 <td class="gamesemcolumn"><a href="DisplayMatch.ASP?matchid=59261&amp;sessionnumber=7" title="Click to view details of this match">F</a></td>
 <td class="aftergw"></td>
 <td class="racolumn1"></td>
 <td class="racolumn1"></td>
 <td align="right" width="4%">_</td>
 <td align="right" width="4%">6